In [1]:
import pandas as pd
import numpy as np

In [2]:
df_s = pd.read_csv('inference_preembed_result.csv',sep=',')

In [2]:
from sklearn.metrics import accuracy_score

In [3]:
def getGender(x):
    if x>0.5:
        return 1
    return 0

In [14]:
df_all_gender.head()

,True_label,Predicted_label,ReviewerID,gender
0,1,1,AUCXT9K30SHYF,1
1,1,0,AUCXT9K30SHYF,1
2,1,1,AUCXT9K30SHYF,1
3,1,0,AUCXT9K30SHYF,1
4,1,1,AUCXT9K30SHYF,1


In [4]:
from sklearn.metrics import confusion_matrix

In [5]:
def precisionRecallF1(conf_mat): 
    
    male_pr = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[1][0])
    male_r = conf_mat[0][0]/(conf_mat[0][0]+conf_mat[0][1])
    
    female_pr = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[0][1])
    female_r = conf_mat[1][1]/(conf_mat[1][1]+conf_mat[1][0])
    
    male_f1 = 2*(male_pr*male_r)/(male_pr + male_r)
    female_f1 = 2*(female_pr*female_r)/(female_pr + female_r)
    
    return male_pr, male_r, male_f1, female_pr, female_r, female_f1
    

In [6]:
def evaluation(fname): # input: csv file, calculates accuracy, f1-score w/o majority voting
    
    df_s = pd.read_csv(fname)
    
    df_gender = df_s.groupby('ReviewerID').agg({'Predicted_label':'sum','True_label':'count'}). \
reset_index().rename(columns={'Predicted_label':'label_sum','True_label':'l_count'})
    
    df_gender['gender'] = df_gender['label_sum']/df_gender['l_count']
    
    df_gender['gender'] = df_gender['gender'].apply(lambda x:getGender(x))
    
    df_gender = df_gender.drop(columns=['label_sum','l_count'])
    
    df_all_gender = pd.merge(df_s,df_gender,on='ReviewerID')
    
    acc = accuracy_score(df_all_gender.True_label,df_all_gender.Predicted_label) 
    
    acc_mv = accuracy_score(df_all_gender.True_label,df_all_gender.gender)
    
    conf_mat_mv = confusion_matrix(df_all_gender.True_label,df_all_gender.gender)
    
    conf_mat = confusion_matrix(df_all_gender.True_label,df_all_gender.Predicted_label)
    
    male_pr_mv, male_r_mv, male_f1_mv, female_pr_mv, female_r_mv, female_f1_mv = precisionRecallF1(conf_mat_mv)
    
    male_pr, male_r, male_f1, female_pr, female_r, female_f1 = precisionRecallF1(conf_mat)
    
    print('without majority voting results -')
    print(f'accuracy: {acc}, male f1: {male_f1}, female f1: {female_f1}')
    
    print('with majority voting results -')
    print(f'accuracy: {acc_mv}, male f1: {male_f1_mv}, female f1: {female_f1_mv}')
    

In [43]:
evaluation('inference_preembed_result.csv')

without majority voting results -
accuracy: 0.7251947642525259, male f1: 0.7120841051861675, female f1: 0.7371634019328063
with majority voting results -
accuracy: 0.8012269391087037, male f1: 0.7965013105056868, female f1: 0.8057380724414493


In [11]:
evaluation('inference_result_RNN_vanilla_2.csv')

without majority voting results -
accuracy: 0.7336565572632477, male f1: 0.7218600729240505, female f1: 0.7444931273779037
with majority voting results -
accuracy: 0.8109789833112898, male f1: 0.8071954614496337, female f1: 0.8146168703093878


In [12]:
evaluation('../HAN/output_han_amazon.csv')

without majority voting results -
accuracy: 0.7254032684854282, male f1: 0.7171541336853197, female f1: 0.7331848705234507
with majority voting results -
accuracy: 0.8025948600332903, male f1: 0.8013855659988811, female f1: 0.803789517246777


In [8]:
evaluation('../amazonUser/output_Char_cnn_2.csv')

without majority voting results -
accuracy: 0.7520313359329179, male f1: 0.7493713654955585, female f1: 0.7546354377477685
with majority voting results -
accuracy: 0.8211777201203803, male f1: 0.82277158711336, female f1: 0.8195549247411497


In [7]:
evaluation('output_Char_cnn_2.csv')

without majority voting results -
accuracy: 0.7520313359329179, male f1: 0.7493713654955585, female f1: 0.7546354377477685
with majority voting results -
accuracy: 0.8211777201203803, male f1: 0.82277158711336, female f1: 0.8195549247411497
